# TG bot 

In [3]:
#!pip install -Uq telethon 
# see https://docs.telethon.dev/en/stable/basic/installation.html#optional-dependencies
#!pip install -Uq telethon cryptg pillow
#!pip install -Uq python-dotenv

In [1]:
from telethon import TelegramClient, events
import asyncio
from dotenv import load_dotenv
import os, json
from pathlib import Path

In [3]:
# For running in Jupyter/IPython
import nest_asyncio
nest_asyncio.apply()

## Development

In [6]:
test_settings = {
    "keywords": ["python", "ai", "machine learning", "data science"],
    "channels": ["ejdailyru", "pythonnews", "fastdotai"]
}

# Write to file
Path('settings.json').write_text(json.dumps(test_settings, indent=2))
data = json.loads(Path('settings.json').read_text())
json.dumps(test_settings)

162

In [20]:
class Settings:
    def __init__(self, filepath='settings.json'):
        self.filepath = Path(filepath)
        self.keywords, self.channels= set(), set()
        self.load()
        
    def load(self):
        if self.filepath.exists(): 
            data = json.loads(self.filepath.read_text())
            self.keywords = set(data.get('keywords',[]))
            self.channels = set(data.get('channels',[]))

    def save(self):
        data = {'keywords': list(self.keywords), 'channels': list(self.channels)}
        self.filepath.write_text(json.dumps(data, indent=2))

In [ ]:
@bot.on(events.NewMessage(pattern='/add_keyword (.+)'))
async def add_keyword(event):
    new_keywords = {k.strip().lower() for k in event.pattern_match.group(1).replace(',', ' ').split()}
    settings.keywords.update(new_keywords)
    settings.save()
    await event.respond(f'Added keywords: {", ".join(new_keywords)}')

In [ ]:
@bot.on(events.NewMessage(pattern='/list_keywords'))
async def list_keywords(event):
    await event.respond(f'Keywords: {", ".join(keywords) or "none"}')

In [7]:
async def monitor_channels(event):
    if any(k.lower() in event.text.lower() for k in keywords):
        # TODO = forward message + highlight found text
        print(f"Keyword found in message: {event.text[:100]}...")

In [8]:
async def update_monitored_channels():
    # Remove existing handler if any
    client.remove_event_handler(monitor_channels)
    if channels:
        # Add new handler with current channels
        client.add_event_handler(monitor_channels, events.NewMessage(chats=list(channels)))

In [ ]:
@bot.on(events.NewMessage(pattern='/add_channel (.+)'))
async def add_channel(event):
    new_channels = {c.strip().lower().replace('@', '') for c 
                    in event.pattern_match.group(1).replace(',', ' ').split()}
    settings.channels.update(new_channels)
    # update monitoring logic
    await update_monitored_channels()
    settings.save()
    await event.respond(f'Added channels: {", ".join("@" + c for c in new_channels)}')

In [ ]:
@bot.on(events.NewMessage(pattern='/list_channels'))
async def list_channels(event):
    await event.respond(f'Monitored channels: {", ".join("@" + c for c in channels) or "none"}')

In [ ]:
load_dotenv()
api_id = os.getenv('TELEGRAM_API_ID')
api_hash = os.getenv('TELEGRAM_API_HASH')
bot_token = os.getenv('BOT_TOKEN')

data = json.loads(Path('settings.json').read_text())
json.dumps(test_settings)
settings = Settings() 

In [15]:
async def main():
    try:
        await client.start()
        await bot.start()
        
        # Initial setup of monitoring
        await update_monitored_channels()
        
        # Run both clients concurrently
        await asyncio.gather(
            client.run_until_disconnected(),
            bot.run_until_disconnected()
        )
      
    except Exception as e:
        print(str(e))

    finally:
        await client.disconnect()
        print("Client disconnected!")
        await bot.disconnect()
        print("Bot disconnected!")

In [16]:
try:
    await main()
except KeyboardInterrupt:
    print("Interrupted! Disconnecting client...")
    #await client.disconnect()
    #await bot.disconnect()

Keyword found in message: **Россия увеличила выпуск сжиженного природного газа (СПГ) на 8% в 2024 году**, [следует](http://vdm...
Keyword found in message: Американский независимый сенатор от штата Вермонт Берни Сандерс, которому на данный момент 83 года, ...
Keyword found in message: Ну и как верить тарологам после этого?...
Client disconnected!
Bot disconnected!


CancelledError: 

In [ ]:
и, вчера, по, от, он, в, сказал, сегодня, на

## Testing

Test getting messages from a specified channel and checking if they contain keywords.

In [6]:
channel_username = 'ejdailyru'
keywords = {'на', 'по', 'и', 'в'}

In [7]:
def contains_keywords(text: str, keywords: list[str]) -> bool:
    return any(keyword.lower() in text.lower() for keyword in keywords)

In [8]:
async def test_get_messages():
    try:
        await client.start()
        # Let's test with a simple message retrieval
        messages = await client.get_messages(channel_username, limit=3)
        for msg in messages: 
            #print(f"{msg.text[:10]}...\n" if msg.text else 'No text')
            if msg.text and contains_keywords(msg.text, keywords): print(f"Found keyword in message: {msg.text[:100]}...")
            
    except Exception as e:
        print(str(e))

    finally:
        await client.disconnect()
        print("Client disconnected!")

In [9]:
# Now try running the main function
try:
    await test_get_messages()
except KeyboardInterrupt:
    print("Interrupted! Disconnecting client...")
    await client.disconnect()

Found keyword in message: Политолог Максим Жаров:
**Кремль готовит страну к усилению санкционного давления при Трампе** Сразу ...
Found keyword in message: Движение йеменских хуситов «Ансар Аллах» уведомило владельцев и операторов коммерческих судов, что с...
Found keyword in message: **Попавшие под санкции танкеры не могут вывезти нефть из крупнейшего порта России на Дальнем Востоке...
Client disconnected!


Test getting new message from a specific channel

In [56]:
@client.on(events.NewMessage(chats=channel_username))
async def handler(event):
    if any(k.lower() in event.text.lower() for k in keywords):
        print(f"Keyword found in message: {event.text[:100]}...")


In [59]:
async def main():
    try:
        await client.start()
        await client.run_until_disconnected()
      
    except Exception as e:
        print(str(e))

    finally:
        await client.disconnect()
        print("Client disconnected!")

In [60]:
# Now try running the main function
try:
    await main()
except KeyboardInterrupt:
    print("Interrupted! Disconnecting client...")
    await client.disconnect()

Keyword found in message: 725 млн долларов готовы выделить США по программе малых грантов «на продвижение демократических инст...
Client disconnected!


CancelledError: 